# Genetic Algorithm

In this project, the aim is to solve a sudoku puzzle using the genetic algorithm

In [1]:
import random as rndm
import numpy as np
import time

## Part 1: Defining Genes and Chromosomes

A gene is the value of a row and is a premutation of set {1 ... 9}. A chromosome consists of 9 genes, each gene representing a row of the actual sudoku puzzle.

In [2]:
def make_gene(initial=[0]*9):
    mapp={}
    gene=list(range(1,10));
    rndm.shuffle(gene);
    for i in range(9):
        mapp[gene[i]]=i
    for i in range(9):
        if initial[i]!=0 and gene[i]!=initial[i]:
            temp=gene[i],gene[mapp[initial[i]]]
            gene[mapp[initial[i]]],gene[i]=temp
            mapp[initial[i]],mapp[temp[0]]=i,mapp[initial[i]]
    return gene;

In [3]:
def make_chromosome(initial=[[0]*9]*9):
    chromosome=[]
    for i in range(9):
        chromosome.append(make_gene(initial[i]));
    return chromosome

## Part 2: Making First Generation

In this part, a function is implemented to randomly create a population.

In [4]:
def make_population(count,initial=[[0]*9]*9):
    population=[]
    for _ in range(count):
        population.append(make_chromosome(initial))
    return population

## Part 3: Fitness Function

In this part, the fitness function is implemented to decide the better chromosomes. The fitness function is defined as follows:Start with fitness=0. For every column or $3\times3$ square, if a number is seen more than once, decrease fitness by number of times than number has been repeated(or (number of times seen) -1).
this fitness function is choosen because:
 - It offers a good evaluation of the current solution, since the ultimate goal is to reach a puzzle with 0 repeated numbers in each column and $3\times3$ square, it shows exactly how close the puzzle is to a complete puzzle.
 - It is not complex so is faster to run and implement and a simple solution is prefered if it offers a good evaluation.
 

In [5]:
def get_fitness(chromosome):
    fitness=0
    for i in range(9):
        seen={}
        for j in range(9):
            if chromosome[j][i] in seen:
                seen[chromosome[j][i]]+=1
            else:
                seen[chromosome[j][i]]=1
        for key in seen :
            fitness-=(seen[key]-1)
    for m in range(3):
        for n in range(3):
            seen={}
            for i in range(3*(n),3*(n+1)):
                for j in range(3*(m),3*(m+1)):
                    if chromosome[j][i] in seen:
                        seen[chromosome[j][i]]+=1
                    else:
                        seen[chromosome[j][i]]=1
            for key in seen :
                fitness-=(seen[key]-1)
    return fitness
    

In [6]:
ch=make_chromosome()
print(get_fitness(ch))
def pch(ch):
    for i in range(9):
        for j in range(9):
            print(ch[i][j],end=" ")
        print("")
    

-55


## Part 4: Crossover and Mutation

In this part, the crossover and mutation function is implemented to help in determining the next generation.

### Crossover

the corssover function takes two chromosomes as input and makes two new chromosomes by combining them. This crossover function decides the parent of each gene seperately, so the result is independent of the location of the genes.

In [7]:
def crossover(ch1,ch2):
    newch1=[]
    newch2=[]
    for i in range(9):
        x=rndm.randint(0,1)
        if(x==1):
            newch1.append(ch1[i])
            newch2.append(ch2[i])
        elif(x==0):
            newch2.append(ch1[i])
            newch1.append(ch2[i])
    return newch1,newch2

### Mutation

In [8]:
def mutation(ch,pm,initial):
    for i in range(9):
        x=rndm.randint(0,100)
        if(x<pm*100):
            ch[i]=make_gene(initial[i])
    return ch

## Part 5: Implementing The Genetic Algorithm

In this part, we use the components defined in previous steps to write the genetic algorithm.

In [9]:
def read_puzzle(address):
    puzzle=[]
    f=open(address,'r')
    for row in f:
        temp=row.split()
        puzzle.append([int(c) for c in temp])
    return puzzle

In [10]:
def r_get_mating_pool(population):
    fitness_list=[]
    pool=[]
    for chromosome in population:
        fitness=get_fitness(chromosome)
        fitness_list.append((fitness,chromosome))
    fitness_list.sort()
    weight=list(range(1,len(fitness_list)+1))
    for _ in range(len(population)):
        ch=rndm.choices(fitness_list,weight)[0]
        pool.append(ch[1])
    return pool
    

In [15]:
def w_get_mating_pool(population):
    fitness_list=[]
    pool=[]
    for chromosome in population:
        fitness=get_fitness(chromosome)
        fitness_list.append((fitness,chromosome))
    weight=[fit[0]-fitness_list[0][0] for fit in fitness_list]
    for _ in range(len(population)):
        ch=rndm.choices(fitness_list,weights=weight)[0]
        pool.append(ch[1])
    return pool
    

In [12]:
def get_offsprings(population,initial,pm,pc):
    new_pool=[]
    i=0
    while i<len(population):
        ch1=population[i]
        ch2=population[(i+1)%len(population)]
        x=rndm.randint(0,100)
        if(x<pc*100):
            ch1,ch2=crossover(ch1,ch2)
        new_pool.append(mutation(ch1,pm,initial))
        new_pool.append(mutation(ch2,pm,initial))
        i+=2
    return new_pool

In [29]:
POPULATION=1000
REPEATATION=1000
PM=0.1
PC=0.95
def genetic_algorithm(initial_file):
    initial=read_puzzle(initial_file)
    population=make_population(POPULATION,initial)
    for _ in range(REPEATATION):
        mating_pool=r_get_mating_pool(population)
        rndm.shuffle(mating_pool);
        population=get_offsprings(mating_pool,initial,PM,PC)
        fit=[get_fitness(c) for c in population]
        m=max(fit)
        if(m==0):
            return population
    return population

In [30]:
tic=time.time()
r=genetic_algorithm("./SampleSudoku/Test2.txt")
toc=time.time()
print("time_taken: ",toc-tic)
fit=[get_fitness(c) for c in r]
m=max(fit)
print(max(fit))
for c in r:
    if get_fitness(c)==m:
        pch(c)
        break

time_taken:  43.68599009513855
0
9 6 8 2 5 3 4 7 1 
4 7 5 1 9 6 3 8 2 
3 1 2 4 8 7 6 9 5 
2 5 1 9 4 8 7 6 3 
7 9 3 6 2 5 8 1 4 
8 4 6 3 7 1 2 5 9 
1 8 7 5 3 2 9 4 6 
6 2 9 8 1 4 5 3 7 
5 3 4 7 6 9 1 2 8 


## Part 6: Questions

<b>1-Method For Selecting Mating Pool:</b> We considered two options for this: rank based and fitness based approach. both were implemented and tested and rank based approach got better results, so it was choosen. rank based approach ranks the chromosomes by their ranks and chooses their weight based on their ranks. Lowest rank has weight 1 and highest rank has weight len(population)

<b>2-Fitness Function:</b> Explained in part3

<b>3-Crossover and Mutation:</b> 

Crossover: It is used to create new genes from old ones until the desired solution is reached, the method for this was explained in part4. This event is most likely to occure because it is the key to reach the solution. In this example, the chance of crossover happening is 95%. This number was determined by trial and error, other candidates were 90% and 85%.

Mutation: It takes a chromosome as input and changes its genes with probability of PM. Sometimes the desired solution cannot be obtained from the current genes, so this function helps by changing the genes and creating new ones. It also can help if the algorithm is stuck at some local minimum. however, the chances of crossover happening is low because it would interfer with main genetic algorithm and made it a totally random algorithm and the algorithm wont converge if mutation was more likely to happen. In this algorithm, the chance of mutation happening is 10%. Same as crossover, this number was determined by trial and error, other candidates were 5% and 1%.

<b>4-Problems with Genetic Algorithm:</b> It is possible for the algorithm to get stuck at local maximum and it can cause a problem if local minimum is not enough to satisfy our need (like in sudoku problem). One way to fix this is to increase the mutation rate from the start(although it has its own setbacks as mentioned in above parts), or define a function to recognize local maximum.(like detecting the repeatation of the same value) and increase the mutation rate in this case. another way is to increase population or the number of times the algorithm runs, another way is to change the definition of fitness function or change mating pool selection method,. In general due to random nature of this algorithm, it is possible to face this problem regardless of our choise of method, sometimes runing the algorithm several times can help.